## working word to animation translation

In [1]:
from flask import Flask, request, jsonify, send_file
from flask_socketio import SocketIO, emit
import csv
import cv2
import os
import numpy as np
import io
import time
from flask_cors import CORS

app = Flask(__name__)
socketio = SocketIO(app, cors_allowed_origins="http://localhost:5173")

csv_folder = './csv_files/'  # Folder containing word CSV files
frame_rate = 30  # Frames per second


# Function to read landmark data from a CSV file
def read_landmark_data(csv_file):
    landmark_data = {}
    with open(csv_file, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)  # Skip header
        for row in csvreader:
            frame = int(row[0])
            landmark_type = row[1]
            index = int(row[2])
            x, y, z = float(row[3]), float(row[4]), float(row[5])
            if frame not in landmark_data:
                landmark_data[frame] = {'pose': [], 'face': [], 'left_hand': [], 'right_hand': []}
            landmark_data[frame][landmark_type].append((x, y, z))
    return landmark_data


# Function to draw landmarks
def draw_landmarks(img, landmarks, connections, color):
    for (i, j) in connections:
        if i < len(landmarks) and j < len(landmarks):
            pt1 = (int(landmarks[i][0] * img.shape[1]), int(landmarks[i][1] * img.shape[0]))
            pt2 = (int(landmarks[j][0] * img.shape[1]), int(landmarks[j][1] * img.shape[0]))
            cv2.line(img, pt1, pt2, color, 2)
    for (x, y, z) in landmarks:
        cv2.circle(img, (int(x * img.shape[1]), int(y * img.shape[0])), 3, color, -1)


pose_connections = [
    (0, 1), (1, 2), (2, 3), (3, 7), (0, 4), (4, 5), (5, 6), (6, 8),
    (9, 10), (11, 12), (11, 13), (13, 15), (15, 17), (12, 14), (14, 16)
]

hand_connections = [
    (0, 1), (1, 2), (2, 3), (3, 4),
    (0, 5), (5, 6), (6, 7), (7, 8),
    (0, 9), (9, 10), (10, 11), (11, 12),
    (0, 13), (13, 14), (14, 15), (15, 16),
    (0, 17), (17, 18), (18, 19), (19, 20)
]

# Function to render the animation for a word
def render_word_animation(word):
    csv_file = os.path.join(csv_folder, f'{word}.csv')
    if not os.path.exists(csv_file):
        print(f"CSV for '{word}' not found!")
        return

    landmark_data = read_landmark_data(csv_file)

    for frame_index in sorted(landmark_data.keys()):
        frame_landmarks = landmark_data[frame_index]
        img = np.zeros((720, 1280, 3), dtype=np.uint8)

        if 'pose' in frame_landmarks:
            draw_landmarks(img, frame_landmarks['pose'], pose_connections, (0, 255, 0))
        if 'face' in frame_landmarks:
            draw_landmarks(img, frame_landmarks['face'], [], (255, 0, 0))
        if 'left_hand' in frame_landmarks:
            draw_landmarks(img, frame_landmarks['left_hand'], hand_connections, (0, 0, 255))
        if 'right_hand' in frame_landmarks:
            draw_landmarks(img, frame_landmarks['right_hand'], hand_connections, (0, 0, 255))

        # Convert the frame to a JPEG blob
        _, jpeg = cv2.imencode('.jpg', img)
        frame_blob = jpeg.tobytes()
        # Emit the frame as a blob to SocketIO client
        socketio.emit('animation_frame', frame_blob)

        time.sleep(1 / frame_rate)  # Control frame rate


# Route to trigger animation through a POST request
@app.route('/animation', methods=['POST'])
def animation():
    data = request.get_json()
    sentence = data.get('sentence', '')

    if not sentence:
        return jsonify({"error": "No sentence provided"}), 400

    # Process the sentence and animate each word
    words = sentence.lower().split()
    for word in words:
        render_word_animation(word)

    # Emit completion status
    socketio.emit('animation_complete', {'status': 'completed'})

    return jsonify({"status": "Animation started"}), 200


# WebSocket event to handle sentence processing
@socketio.on('process_sentence')
def handle_sentence(data):
    sentence = data.get('sentence', '')
    words = sentence.lower().split()

    for word in words:
        render_word_animation(word)

    socketio.emit('animation_complete', {'status': 'completed'})


if __name__ == '__main__':
    CORS(app, origins=["http://localhost:5173"])
    socketio.run(app, host='0.0.0.0', port=5001, allow_unsafe_werkzeug=True)


Werkzeug appears to be used in a production deployment. Consider switching to a production web server instead.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.1.3:5001
Press CTRL+C to quit
127.0.0.1 - - [18/Nov/2024 06:19:43] "GET /socket.io/?EIO=4&transport=polling&t=an0gfsre HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2024 06:19:43] "POST /socket.io/?EIO=4&transport=polling&t=an0p8i2g&sid=442FHA5HUJjD9XrDAAAA HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2024 06:19:43] "GET /socket.io/?EIO=4&transport=polling&t=an0p9kmo&sid=442FHA5HUJjD9XrDAAAA HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2024 06:19:43] "GET /socket.io/?EIO=4&transport=websocket&sid=442FHA5HUJjD9XrDAAAA HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2024 06:19:43] "GET /socket.io/?EIO=4&transport=polling&t=an19b9yf HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2024 06:19:44] "POST /socket.io/?EIO=4&transport=polling&t=an19j8zc&sid=9VJLVZWl7hYWSO9RAAAC HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2024 06:19:44] "GET /socket.io/?EIO=4&transport=polling&t=an19jqd3&sid=9VJLVZWl7hYWSO9RAAAC HTTP/1.1" 200 -
127.0.0.1 - - 

CSV for 'bhauju' not found!


127.0.0.1 - - [18/Nov/2024 06:39:48] "GET /socket.io/?EIO=4&transport=websocket&sid=0sZxWyoR9QZ-fBPPAAAl HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2024 06:39:49] "GET /socket.io/?EIO=4&transport=polling&t=bcve33mk HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2024 06:39:49] "POST /socket.io/?EIO=4&transport=polling&t=bcvedm8s&sid=tTfSkod0MlXxpZ4mAAAn HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2024 06:39:49] "GET /socket.io/?EIO=4&transport=polling&t=bcvedphg&sid=tTfSkod0MlXxpZ4mAAAn HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2024 06:39:50] "GET /socket.io/?EIO=4&transport=websocket&sid=tTfSkod0MlXxpZ4mAAAn HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2024 06:39:50] "GET /socket.io/?EIO=4&transport=polling&t=bcwc1rcp HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2024 06:39:50] "POST /socket.io/?EIO=4&transport=polling&t=bcwc9gd6&sid=qxcQp8JEDnCr3nwKAAAp HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2024 06:39:50] "GET /socket.io/?EIO=4&transport=polling&t=bcwcails&sid=qxcQp8JEDnCr3nwKAAAp HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2024